In [46]:
import pandas as pd

In [47]:
df = pd.read_csv('../MovebankDatasets/e-ObsGPRSHimalayanGriffon-Bhutan-MPIAB/ThangKaarBaen(4185)-4185.csv', delimiter=',')
# df.head()

In [48]:
df = df.drop(['event-id', 'visible', 'timestamp', 'bar:barometric-pressure','data-decoding-software','eobs:activity','eobs:activity-samples','eobs:battery-voltage','eobs:fix-battery-voltage','eobs:horizontal-accuracy-estimate','eobs:key-bin-checksum','eobs:speed-accuracy-estimate','eobs:start-timestamp','eobs:status','eobs:temperature','eobs:type-of-fix','eobs:used-time-to-get-fix','gps:dop','gps:satellite-count','ground-speed','heading','height-above-ellipsoid','import-marked-outlier','mag:magnetic-field-raw-x','mag:magnetic-field-raw-y','mag:magnetic-field-raw-z','orientation:quaternion-raw-w','orientation:quaternion-raw-x','orientation:quaternion-raw-y','orientation:quaternion-raw-z','sensor-type','individual-taxon-canonical-name','tag-local-identifier','individual-local-identifier','study-name','utm-easting','utm-northing','utm-zone','study-timezone','study-local-timestamp'], axis=1)
#df.head()

In [49]:
df.rename(columns={'location-long':'longitude'}, inplace = True)
df.rename(columns={'location-lat':'latitude'}, inplace = True)
df = df.sort_values(by =['longitude', 'latitude'])
#df.head()

In [50]:
df.insert(0, 'species', 'griffon_vulture', True)
df.head()

,species,longitude,latitude
143879,griffon_vulture,88.282075,28.840740
143880,griffon_vulture,88.282093,28.840562
143878,griffon_vulture,88.282100,28.840924
143881,griffon_vulture,88.282143,28.840404
143877,griffon_vulture,88.282161,28.841108


### Removing duplicates

In [51]:
data_groups = df.groupby(df.columns.tolist())
size = data_groups.size().reset_index()
len(size[size[0] > 1])   # NUMBER OF DUPLICATES

3573

In [52]:
print(len(df.index))

550110


In [53]:
df = df.drop_duplicates()

In [54]:
print(len(df.index))

544727


### NaN values

In [55]:
df = df.dropna()

In [56]:
print(len(df.index))

544726


### Outliers detection

In [57]:
'''
import plotly.express as px

fig = px.scatter(x=df['latitude'], y=df['longitude'])
fig.show()
'''

"\nimport plotly.express as px\n\nfig = px.scatter(x=df['latitude'], y=df['longitude'])\nfig.show()\n"

### Remove geopoints on distance less than 50

In [72]:
import numpy as np
from math import ceil
from scipy.spatial import distance
from haversine import haversine, Unit

In [75]:
min_distance = 50

print(len(df.index))

count = 1000
iteration_count = ceil(len(df.index) / count)
idx_out = list()

for i in range(iteration_count):
    lower_bound = i * count
    mini_df = df.iloc[lower_bound:lower_bound + count,1:3]
    coords = mini_df.values

    distance_matrix = distance.cdist(coords, coords, lambda u, v: haversine(u, v, Unit.METERS)) # haversine(lat, lon, Unit.Meters)
    n_proxim = np.apply_along_axis(lambda x: np.count_nonzero(x < min_distance), 0, distance_matrix)
    idx = np.argsort(n_proxim).tolist()

    for i in idx:
        for j in range(i+1):
            if i not in idx_out:
                if distance_matrix[i, j] < min_distance:
                    if i != j:
                        idx_out.append(j)

pop_idx = sorted(np.unique(idx_out).tolist(), reverse=True)
df = df.drop(df.index[pop_idx])

print(len(df.index))

543799
2943
3275
3716
4021
4767
542822
